In [38]:
from torchvision import datasets, models, transforms
import dill as pickle
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from utililties import MyDataset
import torch

nrows = 256
ncolumns = 256

dbfile = open('sample.pickle', 'rb')      
db = pickle.load(dbfile) 

image_path = f"data/training/COVID/2020.01.24.919183-p27-132.png"

img = cv2.imread(image_path)
t = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

root_folder = os. getcwd() +'/data/training'
import pickle
from glob import glob

diction = {}

from random import shuffle

# dbfile = open('sample.pickle', 'rb')      
# db_test = pickle.load(dbfile) 


transform = transforms.Compose([
        transforms.Lambda(lambda x: x/255),
        transforms.ToPILImage(), 
        transforms.Resize((230, 230)),
        transforms.RandomResizedCrop((224),scale=(0.75,1.0)),     
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.45271412, 0.45271412, 0.45271412],
                             [0.33165374, 0.33165374, 0.33165374])])



def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

# matplotlib_imshow(t)

data = torch.from_numpy(t).float()
# print(data)
print (data.shape)
X = []
if t.ndim>=3:
    X.append(np.moveaxis(cv2.resize(t[:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
else:
    smimg= cv2.cvtColor(t,cv2.COLOR_GRAY2RGB)
    X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))

x = np.array(X)

print(X)




torch.Size([551, 725, 3])
[array([[[ 43,  43,  47, ...,  45,  43,  39],
        [ 45,  37,  43, ...,  44,  41,  43],
        [ 44,  45,  39, ...,  40,  41,  42],
        ...,
        [ 58,  53,  49, ..., 168, 252, 231],
        [ 48,  55,  57, ..., 246, 238, 129],
        [ 61,  57,  57, ..., 247, 148,  78]],

       [[ 43,  43,  47, ...,  45,  43,  39],
        [ 45,  37,  43, ...,  44,  41,  43],
        [ 44,  45,  39, ...,  40,  41,  42],
        ...,
        [ 58,  53,  49, ..., 168, 252, 231],
        [ 48,  55,  57, ..., 246, 238, 129],
        [ 61,  57,  57, ..., 247, 148,  78]],

       [[ 43,  43,  47, ...,  45,  43,  39],
        [ 45,  37,  43, ...,  44,  41,  43],
        [ 44,  45,  39, ...,  40,  41,  42],
        ...,
        [ 58,  53,  49, ..., 168, 252, 231],
        [ 48,  55,  57, ..., 246, 238, 129],
        [ 61,  57,  57, ..., 247, 148,  78]]], dtype=uint8)]
